In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import dateparser
from MeesmanScraper import config as cfg

In [10]:
conf = cfg.get_configuration()
user_meesman = conf['Meesman'].get('user')
pw_meesman = conf['Meesman'].get('password')
host_mysql = conf['MySQL'].get('host')
user_mysql = conf['MySQL'].get('user')
pw_mysql = conf['MySQL'].get('password')
db_mysql = conf['MySQL'].get('database')

In [67]:
browser = webdriver.Chrome()
browser.get('https://mijn.meesman.nl/login')

In [68]:
username = browser.find_element_by_id('username')
username.send_keys(user_meesman)

In [69]:
password = browser.find_element_by_id('password')
password.send_keys(pw_meesman)

In [70]:
login = browser.find_element_by_xpath('//*[@id="login-submit"]')
login.click()

In [71]:
portefeuille = browser.find_element_by_link_text('Portefeuille')
portefeuille.click()

In [73]:
table = browser.find_element_by_class_name('meesmantable')
table_html = table.get_attribute('outerHTML')
df = pd.read_html(table_html, thousands='.', decimal=',')
meesmantable = df[0].dropna()

In [75]:
browser.close()

In [76]:
meesmantable = meesmantable.assign(Datum=meesmantable.Datum.map(lambda x: dateparser.parse(x, languages=['nl'])))

In [77]:
meesmantable['Koers'] = meesmantable.Koers.str.replace('[€/s.]', '').str.replace(',', '.').astype(float)

In [78]:
meesmantable['Waarde'] = meesmantable.Waarde.str.replace('[€/s.]', '').str.replace(',', '.').astype(float)

In [83]:
meesmantable['Actuele weging'] = meesmantable['Actuele weging'].str.replace('\%', '').astype(int) / 100

In [93]:
meesmantable

,Fonds,Aantal,Koers,Datum,Waarde,Actuele weging
1,Aandelen Wereldwijd,34.2931,111.8083,2018-10-12,3834.25,0.70
2,Aandelen Opkomende Landen,11.3441,75.3263,2018-10-12,854.51,0.16
5,Obligaties Wereldwijd,15.4031,51.1995,2018-10-12,788.63,0.14


In [85]:
from sqlalchemy import create_engine

In [87]:
engine = create_engine('mysql://{}:{}@{}/{}'.format(user_mysql, pw_mysql,
                                                    host_mysql, db_mysql))

In [88]:
con = engine.connect()

In [90]:
meesmantable.to_sql('portefeuille', con = con, if_exists='append', index=False, )

In [91]:
con.close()

In [92]:
engine.has_table('portefeuille')

True